In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from PIL import Image
import numpy as np
import os
import pathlib
import cv2

In [2]:
#img_gen = ImageDataGenerator(validation_split=0.3,rescale=1/255,dtype='float32')

#load images and labels
img_gen = ImageDataGenerator(dtype='float32')
train_image_gen = img_gen.flow_from_directory("eyes",target_size=(64,64),color_mode='grayscale',batch_size=1)

images_array = train_image_gen[0][0]
labels_array = train_image_gen[0][1]
for i in range(1,2423):
    images_array = np.append(images_array,train_image_gen[i][0])
    labels_array = np.append(labels_array,train_image_gen[i][1])

#print(labels_array)
#print(images_array)

Found 2423 images belonging to 2 classes.


In [3]:
#images preprocessing
images_array = images_array.reshape(2423,64,64,1)

images_array = images_array.astype('float32')

images_array = (images_array - 128.0) / 128 #range of input date become [-1:1]

In [4]:
#label preprocessing
labels_done = []
labels_array = labels_array.reshape(2423,2)
print(labels_array)

for i in range(len(labels_array)):
    for j in range(0,2):
        if(labels_array[i][j]):
            labels_done = np.append(labels_done,j)

test_labels = labels_done

labels_done = to_categorical(labels_done, 2, dtype = 'float32')
print(labels_done.shape)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
(2423, 2)


In [5]:
print(labels_done.shape)
print(images_array.shape)

(2423, 2)
(2423, 64, 64, 1)


In [6]:
img = cv2.imread("eyes/ClosedFace/closed_eye_0001.jpg_face_1.jpg")
img.shape

(100, 100, 3)

In [7]:
input_shape=(64,64,1)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),   # 1st layer
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu'),                                        # 2nd layer
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),                                        # 3rd layer
    MaxPooling2D(pool_size=(2, 2)),
#    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
#    Dropout(0.5),
    Dense(2, activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

In [9]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
print(train_image_gen.class_indices)
num_steps_per_epoch = len(train_image_gen) ##batch_size
print(num_steps_per_epoch)

{'ClosedFace': 0, 'OpenFace': 1}
2423


In [11]:
model.fit(images_array,labels_done,epochs = 10,steps_per_epoch=num_steps_per_epoch)

Epoch 1/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.4539 - accuracy: 0.7875
Epoch 2/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.1820 - accuracy: 0.9265
Epoch 3/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.1416 - accuracy: 0.9439
Epoch 4/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.1077 - accuracy: 0.9596
Epoch 5/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.0874 - accuracy: 0.9653
Epoch 6/10
2423/2423 [==============================] - 13s 6ms/step - loss: 0.0820 - accuracy: 0.9707
Epoch 7/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.0696 - accuracy: 0.9802
Epoch 8/10
2423/2423 [==============================] - 13s 5ms/step - loss: 0.0563 - accuracy: 0.9802
Epoch 9/10
2423/2423 [==============================] - 14s 6ms/step - loss: 0.0633 - accuracy: 0.9781
Epoch 10/10
2423/2423 [==============================] - 14s 6ms/step - l

In [12]:
# Save weights of this model
model.save_weights('eyes_model.h5')

In [13]:
#load weights to this TensorFlow model
model.load_weights('eyes_model.h5')

In [14]:

#convert model into TFLM format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

#preprocess representative images
preprocessed_test_images = images_array
preprocessed_test_images = tf.cast(preprocessed_test_images, tf.float32)
tflite_ds = tf.data.Dataset.from_tensor_slices((preprocessed_test_images)).batch(1) #construct a dataset 

def representative_data_gen():
    for input_value in tflite_ds.take(100):
        yield [input_value]
    
converter.representative_dataset = representative_data_gen

#convert model
converted_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\Japoka\AppData\Local\Temp\tmp8rl0eucs\assets


In [15]:
import pathlib
generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"eyes_model_int8.tflite"
converted_model_file.write_bytes(converted_model)

333184

In [16]:
##Evaluate the Model

In [17]:
interpreter = tf.lite.Interpreter(model_path=str(converted_model_file))
interpreter.allocate_tensors()

In [23]:
max_samples = 500
input_scale, input_zero_point = interpreter.get_input_details()[0]['quantization']
print(input_scale,input_zero_point)
output_scale, output_zero_point = interpreter.get_output_details()[0]['quantization']
print(output_scale,output_zero_point)

0.0078125 0
0.00390625 -128


In [24]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    scale, zero_point = interpreter.get_input_details()[0]['quantization']

    prediction_values = []
    output_buffer = []
    #only consider the result having strong confidence(?%) to reduce lossing accuracy
    higher_threshold = 0
    lowwer_threshold = 0
    
    for test_image in preprocessed_test_images[:max_samples]:
        # Pre-processing: add batch dimension, quantize and convert inputs to int8 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0) #.astype(np.float32)
        test_image = np.int8(test_image / scale + zero_point)
        interpreter.set_tensor(input_index, test_image)

        interpreter.invoke()

        # Find the answer with highest probability
        output = interpreter.tensor(output_index)
        output_buffer = np.append(output_buffer,output()[0][1])
        result = np.argmax(output()[0])
        prediction_values.append(result)
        
    accurate_count_k = 0
    num_data_used = 0
    
    for index in range(len(prediction_values)):
        if(output_buffer[index]>higher_threshold or output_buffer[index]<lowwer_threshold):
            num_data_used +=1
            if prediction_values[index] == test_labels[index]:
                accurate_count_k +=1
                
    #print(num_data_used)    

    accuracy = accurate_count_k * 1.0 / num_data_used
    
    return accuracy * 100

In [25]:
print(str(evaluate_model(interpreter)) + "%")
#evaluate_model(interpreter)

98.19277108433735%
